In [1]:
import os

In [3]:
os.environ["HOST"] = "localhost"
os.environ["BBUY_DATA"] = "/Users/krzysztof.mioduszews/Development/Courses/corise-search-eng/datasets/product_data/products"
os.environ["WEEK1_PATH"] = "/Users/krzysztof.mioduszews/Development/Courses/corise-search-eng/search_engineering/week1/"
os.environ["WEEK2_PATH"] = "/Users/krzysztof.mioduszews/Development/Courses/corise-search-eng/search_engineering/week2/"


In [5]:
!curl -k -X PUT -u admin:admin "https://localhost:9200/bbuy_products" -H 'Content-Type: application/json' -d @$WEEK1_PATH/bbuy_products.json

{"acknowledged":true,"shards_acknowledged":true,"index":"bbuy_products"}

In [8]:
!python index.py -s $BBUY_DATA -w 16 -b 500

INFO:Indexing /Users/krzysztof.mioduszews/Development/Courses/corise-search-eng/datasets/product_data/products to bbuy_products with 16 workers, refresh_interval of -1 to host localhost with a maximum number of docs sent per file per worker of 200000 and 500 per batch.
^C

Caught SIGINT. Shutting down workers...



Indexed 675577 documents (~50%) in 8m 37.1s. Introducing -m (as suggested by Dave) to limit number of files to spend less time indexing.

In [12]:
# clean up first
!curl -k -X DELETE -u admin:admin https://localhost:9200/bbuy_products


{"acknowledged":true}

In [13]:
!curl -k -X PUT -u admin:admin "https://localhost:9200/bbuy_products" -H 'Content-Type: application/json' -d @$WEEK1_PATH/bbuy_products.json

{"acknowledged":true,"shards_acknowledged":true,"index":"bbuy_products"}

In [14]:
!python index.py -s $BBUY_DATA -w 16 -b 500 -m 1000

INFO:Indexing /Users/krzysztof.mioduszews/Development/Courses/corise-search-eng/datasets/product_data/products to bbuy_products with 16 workers, refresh_interval of -1 to host localhost with a maximum number of docs sent per file per worker of 1000 and 500 per batch.
INFO:Done. 255077 were indexed in 2.909982067916667 minutes.  Total accumulated time spent in `bulk` indexing: 35.346926625566674 minutes


![Documents Indexed Rate](documents_indexed_rate_l1.png)

Re-indexing

In [15]:
!python index.py -s $BBUY_DATA -w 16 -b 500 -m 1000

INFO:Indexing /Users/krzysztof.mioduszews/Development/Courses/corise-search-eng/datasets/product_data/products to bbuy_products with 16 workers, refresh_interval of -1 to host localhost with a maximum number of docs sent per file per worker of 1000 and 500 per batch.
INFO:Done. 255077 were indexed in 2.821483982833333 minutes.  Total accumulated time spent in `bulk` indexing: 34.966568804966656 minutes


In [17]:
os.environ["BBUY_QUERIES"] = "/Users/krzysztof.mioduszews/Development/Courses/corise-search-eng/datasets/"

In [ ]:
!python query.py -q $BBUY_QUERIES/train.csv -w 4 -m 25000


```bash
# truncated output

INFO:Loading query file from /Users/krzysztof.mioduszews/Development/Courses/corise-search-eng/datasets//train.csv and using seed 84 for worker: 1
INFO:Loading query file from /Users/krzysztof.mioduszews/Development/Courses/corise-search-eng/datasets//train.csv and using seed 126 for worker: 2
INFO:Loading query file from /Users/krzysztof.mioduszews/Development/Courses/corise-search-eng/datasets//train.csv and using seed 168 for worker: 3
INFO:Loading query file from /Users/krzysztof.mioduszews/Development/Courses/corise-search-eng/datasets//train.csv and using seed 42 for worker: 0
INFO:WN: 0: Running queries, checking in every 1000 queries:
INFO:WN: 1: Running queries, checking in every 1000 queries:
INFO:WN: 2: Running queries, checking in every 1000 queries:
INFO:WN: 3: Running queries, checking in every 1000 queries:

# ...

^C

Caught SIGINT. Shutting down workers...

WARNING:WN: 0: Failed to process query: Switchfoot
WARNING:WN: 3: Failed to process query: OnlineMidnightSale_Car_GPS
WARNING:WN: 1: Failed to process query: 2398896 2402035 5386263 5386272 6804112 8579932 8589878 9374278 9650424
WARNING:WN: 2: Failed to process query: car system
INFO:WN: 0: Stopped early.
INFO:WN: 3: Stopped early.
INFO:WN: 1: Stopped early.
INFO:WN: 2: Stopped early.
INFO:Query worker finished in time: 8.79507190305
INFO:Query worker finished in time: 8.795107660833335
INFO:Query worker finished in time: 8.795038516983334
INFO:Query worker finished in time: 8.795067490816667
```

Interrupted above after 8 minutes - observed similar bahaviour as in project meterials.
![Search Query Rate](search_query_rate_l1.png)
![Thread Pools](thread_pools_and_the_jvm_1.png)


## Level 2

### CPU 1 -> 2

In [20]:
!python index.py -s $BBUY_DATA -w 16 -b 500 -m 1000

INFO:Indexing /Users/krzysztof.mioduszews/Development/Courses/corise-search-eng/datasets/product_data/products to bbuy_products with 16 workers, refresh_interval of -1 to host localhost with a maximum number of docs sent per file per worker of 1000 and 500 per batch.
INFO:Done. 255077 were indexed in 2.7120400490833334 minutes.  Total accumulated time spent in `bulk` indexing: 22.36851046428335 minutes


A bit quicker, we're going above 1.5K, but not much.
![documents_indexed_rate_l2_cpu2.png](documents_indexed_rate_l2_cpu2.png)

In [ ]:
!python query.py -q $BBUY_QUERIES/train.csv -w 4 -m 25000

In [ ]:
# Truncated

Intersting results. Considerable boost to performance - almost 3x QPS with 3 threads active.

![QPS L2 2 CPU](search_query_rate_l2_cpu2.png)
![Threads L2 2 CPU](thread_pools_and_the_jvm_l2_cpu2.png)

### CPU 2 -> 4

In [22]:
!python index.py -s $BBUY_DATA -w 16 -b 500 -m 1000

INFO:Indexing /Users/krzysztof.mioduszews/Development/Courses/corise-search-eng/datasets/product_data/products to bbuy_products with 16 workers, refresh_interval of -1 to host localhost with a maximum number of docs sent per file per worker of 1000 and 500 per batch.
INFO:Done. 255077 were indexed in 2.3615369848666665 minutes.  Total accumulated time spent in `bulk` indexing: 19.266531507616673 minutes


Once again quicker, but not groundbreaking. 2k DPS more easily achieved.

![DPS L2 CPU 4](dps_l2_cpu4.png)

In [ ]:
!python query.py -q $BBUY_QUERIES/train.csv -w 4 -m 25000

In [24]:
# Truncated

A bit of improvement, we're going above 200 QPS. Overall it's not that dramatic as in CPU 1 -> CPU 2 case. It seems that GC is blocked whole thread pool (4 threads) utilization.

![QPS L2 CPU 4](qps_l2_cpu4.png)
![Thread L2 CPU 4](threads_l2_cpu4.png)